In [86]:
import requests
import pandas as pd
import json
import sys
from pathlib import Path
import xlrd
import csv
import os
import tempfile
import numpy as np
from typing import Optional

In [17]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

/home/ben/Projects/Graphs4GoodHackathon/ProjectDomino/.Domino/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
with open("config.json") as f:
    config = json.load(f)
    for key in config:
        os.environ[key] = config[key]

In [21]:
def xlsHandler(r):
    df = pd.DataFrame()
    with tempfile.NamedTemporaryFile("wb") as xls_file:
        xls_file.write(r.content)
    
        try:
            book = xlrd.open_workbook(xls_file.name,encoding_override="utf-8")  
        except:
            book = xlrd.open_workbook(xls_file.name,encoding_override="cp1251")

        sh = book.sheet_by_index(0)
        with tempfile.NamedTemporaryFile("w") as csv_file:
            wr = csv.writer(csv_file, quoting=csv.QUOTE_ALL)

            for rownum in range(sh.nrows):
                wr.writerow(sh.row_values(rownum))
            df = pd.read_csv(csv_file.name)
            csv_file.close()

        xls_file.close()
    return df

In [39]:
def csvZipHandler(r):
    df = pd.DataFrame()
    with tempfile.NamedTemporaryFile("wb",suffix='.csv.zip') as file:
        file.write(r.content)
        df = pd.read_csv(file.name)
        file.close()
    return df

In [22]:
def urlToDF(url:str,respHandler) -> pd.DataFrame:
    r = requests.get(url, allow_redirects=True)
    df = pd.DataFrame()
    return respHandler(r)

In [23]:
def api(query,from_study,to_study):
    url = os.environ["URL_USA"].format(query,from_study,to_study)
    response = requests.request("GET", url)
    return response.json()

In [24]:
def apiWrapper(query,from_study):
    return api(query,from_study,from_study+99)

In [25]:
def getAllStudiesByQuery(query:str) -> list:
    studies:list = []
    from_study = 1
    temp = apiWrapper(query,from_study)
    nstudies = temp['FullStudiesResponse']['NStudiesFound']
    print("> {} studies found by '{}' keyword".format(nstudies,query))
    if nstudies > 0:
        studies = temp['FullStudiesResponse']['FullStudies']
        for study_index in range(from_study+100,nstudies,100):
            temp = apiWrapper(query,study_index)
            studies.extend(temp['FullStudiesResponse']['FullStudies'])
    
    return studies

In [27]:
url_int = os.environ["URL_INT"]

In [31]:
internationalstudies = urlToDF(url_int,xlsHandler)

In [29]:
internationalstudies.columns

Index(['TrialID', 'Last Refreshed on', 'Public title', 'Scientific title', 'Acronym', 'Primary sponsor', 'Date registration', 'Date registration3', 'Export date', 'Source Register', 'web address', 'Recruitment Status', 'other records', 'Inclusion agemin', 'Inclusion agemax', 'Inclusion gender', 'Date enrollement', 'Target size', 'Study type', 'Study design', 'Phase', 'Countries', 'Contact Firstname', 'Contact Lastname', 'Contact Address', 'Contact Email', 'Contact Tel', 'Contact Affiliation', 'Inclusion Criteria', 'Exclusion Criteria', 'Condition', 'Intervention', 'Primary outcome', 'results date posted', 'results date completed', 'results url link', 'Retrospective flag', 'Bridging flag truefalse', 'Bridged type', 'results yes no'], dtype='object')

In [30]:
internationalstudies["Intervention"]

0      survival group:none;died:none;                                                                                                                                                                                                                                 
1      Case series:Treated with conventional treatment combined with Chloroquine Phosphate;                                                                                                                                                                           
2      experimental group:standardized comprehensive treatment combined with convalescent plasma treatment;control group:standardized comprehensive treatment;                                                                                                        
3      control group:Western Medicine;experimental group:Integrated Traditional Chinese and Western Medicine;                                                                                                      

In [44]:
all_US_studies_by_keyword:dict = {}
queries:list = ["covid-19", "SARS-CoV-2", "coronavirus"]

for key in queries:
    all_US_studies_by_keyword[key] = getAllStudiesByQuery(key)

> 318 studies found by 'covid-19' keyword
> 318 studies found by 'SARS-CoV-2' keyword
> 288 studies found by 'coronavirus' keyword


In [47]:
print(len(all_US_studies_by_keyword))
with open('all_US_studies_by_keyword.json', 'w', encoding='utf-8') as f:
    json.dump(all_US_studies_by_keyword, f, ensure_ascii=False, indent=4)

3


In [49]:
drug_url = os.environ["URL_DRUGBANK"]
vocab = urlToDF(drug_url,csvZipHandler) 

In [50]:
vocab.columns

Index(['DrugBank ID', 'Accession Numbers', 'Common name', 'CAS', 'UNII', 'Synonyms', 'Standard InChI Key'], dtype='object')

In [43]:
vocab.head(20)

,DrugBank ID,Accession Numbers,Common name,CAS,UNII,Synonyms,Standard InChI Key
0,DB00001,BIOD00024 | BTD00024,Lepirudin,138068-37-8,Y43GF64R34,Hirudin variant-1 | Lepirudin recombinant,NaN
1,DB00002,BIOD00071 | BTD00071,Cetuximab,205923-56-4,PQX0D8J21J,"Cetuximab | Cétuximab | Cetuximabum | Immunoglobulin G 1 (human-mouse monoclonal C 225 gamma 1 - chain anti-human epidermal growt factor receptor), disulfide wit human-mouse monoclonal C 225 kappa - chain, dimer",NaN
2,DB00003,BIOD00001 | BTD00001,Dornase alfa,143831-71-4,953A26OA1Y,"Deoxyribonuclease (human clone 18-1 protein moiety) | Dornasa alfa | Dornase alfa, recombinant | Dornase alpha | Recombinant deoxyribonuclease (DNAse)",NaN
3,DB00004,BIOD00084 | BTD00084,Denileukin diftitox,173146-27-5,25E79B5CTM,Denileukin | Interleukin-2/diptheria toxin fusion protein,NaN
4,DB00005,BIOD00052 | BTD00052,Etanercept,185243-69-0,OP401G7OJC,Etanercept | etanercept-szzs | etanercept-ykro | Recombinant human TNF | rhu TNFR:Fc | rhu-TNFR:Fc | TNFR-Immunoadhesin,NaN
5,DB00006,BIOD00076 | BTD00076 | DB02351 | EXPT03302,Bivalirudin,128270-60-0,TN9BEX005G,Bivalirudin | Bivalirudina | Bivalirudinum,OIRCOABEOLEUMC-GEJPAHFPSA-N
6,DB00007,BIOD00009 | BTD00009,Leuprolide,53714-56-0,EFY6W0M8TG,Leuprorelin | Leuprorelina | Leuproreline | Leuprorelinum,GFIJNRVAKGFPGQ-LIJARHBVSA-N
7,DB00008,BIOD00043 | BTD00043,Peginterferon alfa-2a,198153-51-4,Q46947FE7K,PEG-IFN alfa-2A | PEG-Interferon alfa-2A | Peginterferon alfa-2a | Pegylated Interfeaon alfa-2A | Pegylated interferon alfa-2a | Pegylated interferon alpha-2a | Pegylated-interferon alfa 2a,NaN
8,DB00009,BIOD00050 | BTD00050,Alteplase,105857-23-6,1RXS4UE564,"Alteplasa | Alteplase (genetical recombination) | Alteplase, recombinant | Alteplase,recombinant | Plasminogen activator (human tissue-type protein moiety) | rt-PA | t-PA | t-plasminogen activator | Tissue plasminogen activator | Tissue plasminogen activator alteplase | Tissue plasminogen activator, recombinant | tPA",NaN
9,DB00010,BIOD00033 | BTD00033,Sermorelin,86168-78-7,89243S03TE,NaN,NaN


In [52]:
#vocab.to_csv("vocab.csv")

In [62]:
vocab_red = vocab[['Common name', 'Synonyms']]

In [63]:
drug_vocab:dict = {}

In [80]:
for index, row in vocab_red.iterrows():
    drug_vocab[row['Common name']] = row["Synonyms"].split("|") if isinstance(row["Synonyms"],str) else row["Synonyms"]

In [81]:
from neo4j import GraphDatabase

In [83]:
class DrugSynonimDataToNeo4j(object):

    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close()
    
    def upload_drugs_and_synonims(self,drug_vocab):
        with self._driver.session() as session:
            for key in drub_vocab.keys():
                for synonym in drug_vocab[key]:
                    
            resp = session.write_transaction(self._merge_node, message)
            print(resp) 
    
    @staticmethod
    def _merge_node(tx, node_type, properties=None):
        data:dict = {
            "node_type":node_type
            "properties":self._dict_to_property_str(properties)
        }
        # '{first} {last}'.format(**data)
        base_cypher = """
        MERGE (n:{node_type}) {{ {properties} }})
        RETURN id(n)
        """
        result = tx.run(base_cypher.format(**data))
        
        return result
    
    @staticmethod
    def _merge_edge(tx, from_id, to_id, ):
        result = tx.run("CREATE (a:Greeting) "
                "SET a.message = $message "
                "RETURN a.message + ', from node ' + id(a)", message=message)
        return result
    
    @staticmethod
    def _dict_to_property_str(properties:Optional[dict] = None) -> str:
        def property_type_checker(property_value):
            if isinstance(property_value,int) or isinstance(property_value,float):
                pass
            elif isinstance(property_value,str):
                property_value = """'""" + property_value + """'"""
            return property_value

        resp = ""
        if not properties:
            resp = "{"
            for key in properties.keys():
                resp += """{key}:{value},""".format(key=key,value=property_type_checker(properties[key]))
            resp = resp[:-1] + "}"
        return resp

In [88]:
test:dict = {
    "a":"B",
    "c":"D",
}

In [90]:
asd = "asd,"

In [91]:
print(asd[:-1]+"}")

asd}
